In [6]:
import bs4
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 드라이버 위치 설정

driver_loc = "d:/chromedriver/chromedriver.exe"
# 드라이버 옵션 설정

options = webdriver.ChromeOptions()

options.add_argument("window-size=2280x1440") # 파이썬이 크롬창을 띄울 것인데 창 크기 선택



# 웹 드라이버 정의

driver = webdriver.Chrome(driver_loc, options=options)


enuriBandUrl = "http://www.enuri.com/list.jsp?cate=03181111&tabType=1"

driver.get(enuriBandUrl)


In [5]:

rowList = []

titleList = []

titleList.append('제품명')

titleList.append('최저가')

titleList.append('사이트주소')

for i in range(0, 999) :

    try:

        # 웹페이지 이동 설정
        pageSelector = '#listPagingDiv > li:nth-child('+ str(i + 1) + ') > a'

        page = driver.find_element_by_css_selector(pageSelector)

        page.click()

        time.sleep(0.5)

    except:

        break
    # 웹페이지 소스코드 변수에 저장

    smartBandHtml = driver.page_source


    # 태그 정보만 추출하여 변수에 저장

    bs = bs4.BeautifulSoup(smartBandHtml, 'html.parser')

    divTag = bs.find(name = 'div', attrs = {"class":"goods_list"})

    liTag = divTag.findAll(name = "li", attrs = {"class":"prodItem wide "})



    itemNum = len(liTag)


    columnList = []
    
    for j in range(0, itemNum): 
        
        columnList.append(divTags2[j].find(name = "strong", attrs = {"class":"tit"}).find(name = 'a').text.strip())

        
        driver.find_elements_by_class_name("detailMultiLink prodName")[j].click()



#         bandname =  driver.find_elements_by_class_name('detailMultiLink prodName')[j]
        
        
#         ActionChains(driver).click(bandname).perform()



        time.sleep(3)



        driver.switch_to.window(driver.window_handles[(1)])



        time.sleep(2)

        # 웹페이지 소스코드 변수에 저장

        smartBandHtml2 = driver.page_source



        # 태그 정보만 추출하여 변수에 저장

        bs = bs4.BeautifulSoup(smartBandHtml2, 'html.parser')
        
        if bs.find('div',{'class':'prdcinfo'}) != None :
            dTag = bs.find('div',{'class':'prdcinfo__tit'})
            columnList.append(dTag.find('h3',{'class':"prdcname"}).text)



        if bs.find('div', {'class':'pricebox'}) != None:

            # 최저가 정보 추출

            pTag = bs.find('p', {'class':'priceline'})

            columnList.append(pTag.find('span', {'class':'price'}).text)



            # 최저가 구매사이트 추출

            siteTag = bs.find('span', {'class':'store'})

            columnList.append(siteTag.find('img')['alt'])



        # 테이블 정보를 포함한 상위태그 추출

        tableTag = bs.find('table', {'class':'enuri_spec_table'})

        tbodyTag = tableTag.find('tbody')



        tdTags = tbodyTag.findAll('td')

        thTags = tbodyTag.findAll('th', {'scope':'row'})



        for k in range(0, len(tdTags)):

            if thTags[k].text != '':

                if j == 0 and i == 0:

                    titleList.append(thTags[k].text)



                if k == 0:

                    columnList.append(tdTags[k].text.strip().split(' ')[0])

                else:

                    columnList.append(tdTags[k].text)



        rowList.append(columnList)



        columnList = []



        driver.close()



        driver.switch_to.window(driver.window_handles[0])

# 데이터 가공

## 중복된 데이터 제거를 위해 길이가 82인 row의 7번째 인덱스 삭제

delList = []

rowListLen = len(rowList)

for i in range(0, rowListLen):

    if len(rowList[i]) == 82:

        delList.append(i)



delColNum = 10

for i in delList:

    del rowList[i][delColNum]


## 연주차컬럼 정제. 20XX년 09월 -> 20XX09

yearweekIndex = 4

for i in range(0, len(rowList)):

    rowList[i][yearweekIndex] = rowList[i][yearweekIndex].strip().replace('년 ','').replace('월','')



# 데이터 저장

smartBand = pd.DataFrame(rowList, columns = titleList)

# smartBand.to_csv('smartBand(enuri).csv', index=False)
